In [1]:
import pandas as pd

In [2]:
df=pd.read_csv('train.csv')
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [3]:
###Drop Nan Values
df=df.dropna()

In [4]:
## Get the Independent Features

X=df.drop('label',axis=1)

In [5]:
## Get the Dependent features
y=df['label']

In [6]:
X.shape

(18285, 4)

In [7]:
y.shape

(18285,)

In [8]:
import tensorflow as tf

In [9]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.layers import Dropout

In [10]:
### Vocabulary size
voc_size=5000

# Onehot Representation

In [11]:
messages=X.copy()

In [12]:
messages['title'][1]

'FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'

In [13]:
messages.reset_index(inplace=True)

In [14]:
import nltk
import re
from nltk.corpus import stopwords

In [15]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\nvish\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [16]:
### Dataset Preprocessing
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
#     print(i)
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [17]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [18]:
onehot_repr=[one_hot(words,voc_size)for words in corpus] 
onehot_repr

[[3842, 417, 4184, 2772, 2880, 1534, 4972, 2972, 2226, 2394],
 [2154, 4165, 3054, 3307, 245, 256, 275],
 [4862, 1382, 1781, 3661],
 [851, 661, 2811, 1580, 4593, 2206],
 [2313, 245, 429, 2769, 3908, 1961, 245, 2029, 3237, 1425],
 [4976,
  2457,
  3689,
  972,
  2078,
  1277,
  1446,
  4407,
  1663,
  143,
  3319,
  3707,
  4119,
  860,
  275],
 [2231, 585, 3541, 3462, 1746, 4282, 1117, 4342, 4290, 531, 1166],
 [2353, 2947, 4262, 2925, 4266, 3501, 1277, 3805, 4290, 531, 1166],
 [2016, 2210, 2091, 734, 243, 2768, 1299, 3368, 1277, 2528],
 [620, 4296, 2462, 807, 3221, 1198, 2400, 1353],
 [1545, 248, 2968, 4520, 2184, 3946, 3214, 893, 1738, 3220, 2142],
 [1580, 925, 2880, 2768, 1277, 4266],
 [4286, 3784, 4536, 2599, 898, 2121, 4607, 240, 2069],
 [4883, 3504, 4744, 152, 2054, 1183, 1201, 4290, 531, 1166],
 [2219, 1512, 1707, 1157, 4453, 4290, 531, 1166],
 [2068, 554, 3898, 2648, 1555, 1468, 4449, 1482, 4916, 4214],
 [777, 434, 4165],
 [4634, 2120, 4546, 51, 1277, 2675, 974, 275],
 [2973, 425

# Embedding Representation

In [19]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ... 2972 2226 2394]
 [   0    0    0 ...  245  256  275]
 [   0    0    0 ... 1382 1781 3661]
 ...
 [   0    0    0 ... 4290  531 1166]
 [   0    0    0 ... 4550 2167 3030]
 [   0    0    0 ... 4045 4526 1414]]


In [20]:
embedded_docs[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0, 3842,
        417, 4184, 2772, 2880, 1534, 4972, 2972, 2226, 2394])

## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

In [21]:
## Creating model
embedding_vector_features=40
model1=Sequential()
model1.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model1.add(Bidirectional(LSTM(100)))
model1.add(Dropout(0.3))
model1.add(Dense(1,activation='sigmoid'))
model1.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model1.summary())

W0718 22:33:08.174509  5676 deprecation.py:506] From c:\users\nvish\appdata\local\programs\python\python37\lib\site-packages\tensorflow_core\python\keras\initializers.py:119: calling RandomUniform.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0718 22:33:08.187511  5676 deprecation.py:506] From c:\users\nvish\appdata\local\programs\python\python37\lib\site-packages\tensorflow_core\python\ops\resource_variable_ops.py:1630: calling BaseResourceVariable.__init__ (from tensorflow.python.ops.resource_variable_ops) with constraint is deprecated and will be removed in a future version.
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
W0718 22:33:08.195509  5676 deprecation.py:506] From c:\users\nvish\appdata\local\programs\python\python37\lib\site-packages\tensorflow_core\python\

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 20, 40)            200000    
_________________________________________________________________
bidirectional (Bidirectional (None, 200)               112800    
_________________________________________________________________
dropout (Dropout)            (None, 200)               0         
_________________________________________________________________
dense (Dense)                (None, 1)                 201       
Total params: 313,001
Trainable params: 313,001
Non-trainable params: 0
_________________________________________________________________
None


In [22]:
len(embedded_docs),y.shape

(18285, (18285,))

In [23]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [24]:
X_final.shape,y_final.shape

((18285, 20), (18285,))

In [25]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

In [26]:
### Finally Training
model1.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Train on 12250 samples, validate on 6035 samples
Epoch 1/10
12250/12250 [==============================] - 8s 663us/sample - loss: 0.3111 - acc: 0.8510 - val_loss: 0.2061 - val_acc: 0.9185
Epoch 2/10
12250/12250 [==============================] - 7s 588us/sample - loss: 0.1317 - acc: 0.9499 - val_loss: 0.2195 - val_acc: 0.9143
Epoch 3/10
12250/12250 [==============================] - 7s 583us/sample - loss: 0.0946 - acc: 0.9651 - val_loss: 0.2406 - val_acc: 0.9157
Epoch 4/10
12250/12250 [==============================] - 7s 583us/sample - loss: 0.0647 - acc: 0.9779 - val_loss: 0.3156 - val_acc: 0.9135
Epoch 5/10
12250/12250 [==============================] - 7s 594us/sample - loss: 0.0413 - acc: 0.9852 - val_loss: 0.3487 - val_acc: 0.9117
Epoch 6/10
12250/12250 [==============================] - 7s 592us/sample - loss: 0.0320 - acc: 0.9893 - val_loss: 0.3876 - val_acc: 0.9114
Epoch 7/10
12250/12250 [==============================] - 7s 590us/sample - loss: 0.0183 - acc: 0.9947 - val_lo

# Adding Dropout

from tensorflow.keras.layers import Dropout
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(Dropout(0.3))
model.add(LSTM(100))
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

### Finally Training
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

# Performance Metrics And Accuracy

In [27]:
y_pred=model1.predict_classes(X_test)

In [28]:
from sklearn.metrics import confusion_matrix

In [29]:
confusion_matrix(y_test,y_pred)

array([[3131,  288],
       [ 284, 2332]], dtype=int64)

In [30]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.9052195526097763

In [32]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.92      0.92      0.92      3419
           1       0.89      0.89      0.89      2616

    accuracy                           0.91      6035
   macro avg       0.90      0.90      0.90      6035
weighted avg       0.91      0.91      0.91      6035

